In [ ]:
#create reference for CSV file
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns

In [ ]:
#read CSV file into a dataframe
olympics_path = "resources/athlete_events.csv"

olympics_df = pd.read_csv(olympics_path, low_memory = False)

#print the first 5 rows
olympics_df.head(5)

In [ ]:
# use only the Summer Olympics for Analysis
summer_df = olympics_df[olympics_df['Season'] == 'Summer']

#Show be all the values in the City column
summer_df['City'].unique()

In [ ]:
#Create variables for the specific host country
host_country = 'INSERT HOST COUNTRY HERE'
host_NOC = 'INSERT HOST NOC HERE'
host_year = 'INSERT HOST YEAR HERE'
host_city = 'INSERT HOST CITY HERE'

In [ ]:
# create a dataframe for only the UK team in the Summer Olympics
host_df = summer_df[summer_df['NOC'] == host_NOC]

#View the dataframe
host_df.head(50)

In [ ]:
# create a dataframe for the UK team in the Summer Olympics grouped by Games. 
games_df_host = host_df.groupby('Games').agg({'Medal': ['count']})

# Add columns for gold_medals, silver_medals, bronze_medals and total_medals
games_df_host['gold_medals'] = host_df[host_df['Medal'] == 'Gold'].groupby('Games').agg({'Medal': ['count']})
games_df_host['silver_medals'] = host_df[host_df['Medal'] == 'Silver'].groupby('Games').agg({'Medal': ['count']})
games_df_host['bronze_medals'] = host_df[host_df['Medal'] == 'Bronze'].groupby('Games').agg({'Medal': ['count']})

# fill NaN values with 0
games_df_host = games_df_host.fillna(0)

# format values in all columns except Games to integers
games_df_host = games_df_host.astype(int)

#View the dataframe
games_df_host

In [ ]:
#Fitler and clean the data
std_df = games_df_host

std_df = std_df.rename(columns = {"Medal": "Total Medals"})
std_df = std_df.rename(columns = {"gold_medals": "Gold"})
std_df = std_df.rename(columns = {"silver_medals": "Silver"})
std_df = std_df.rename(columns = {"bronze_medals": "Bronze"})

#Remove the multi-index
std_df.columns = std_df.columns.droplevel(1)

#Move the Total Medals column to the end
std_df = std_df[['Gold', 'Silver', 'Bronze', 'Total Medals']]

#Filter the games to only include the years after 2000
std_df = std_df[std_df.index.str.contains('2004|2008|2012|2016')]

#Reset the index and sort by the column Games in ascending order
std_df = std_df.reset_index()

std_df

In [ ]:
#create new dataframe filtered by columns team, games, year, season, sport, event, medal, city
summer_games_df = summer_df[["Team", "Games", "Year", "Season", "Sport", "Event", "Medal", "City"]]

summer_games_df.head()

In [ ]:
#Clean the Data
#Rename column team to country
summer_games_df = summer_games_df.rename(columns = {"Team": "Country"})
#Rename city to host 
summer_games_df = summer_games_df.rename(columns = {"City": "Host"})

#Filter to the Host Year
summer_games_df = summer_games_df[summer_games_df["Year"] == host_year]

summer_games_df.head()

In [ ]:
#Drop Unnecessary Columns
summer_games_metals_df = summer_games_df.drop(columns = ["Games", "Sport", "Event"])

summer_games_metals_df

In [ ]:
#create a new dataframe thatn removes all rows that have an Na in the medal column
host_year_medals_df = summer_games_metals_df.dropna(subset = ["Medal"]) 

host_year_medals_df

In [ ]:
#create new columns for "Gold" medals, "Silver" medals, "Bronze" medals, and "Total Medals"
host_year_medal_countrys_df = host_year_medals_df.groupby('Country').agg({'Medal': ['count']})

# Add columns for gold_medals, silver_medals, bronze_medals and total_medals
host_year_medal_countrys_df['Gold'] = host_year_medals_df[host_year_medals_df['Medal'] == 'Gold'].groupby('Country').agg({'Medal': ['count']})
host_year_medal_countrys_df['Silver'] = host_year_medals_df[host_year_medals_df['Medal'] == 'Silver'].groupby('Country').agg({'Medal': ['count']})
host_year_medal_countrys_df['Bronze'] = host_year_medals_df[host_year_medals_df['Medal'] == 'Bronze'].groupby('Country').agg({'Medal': ['count']})

# fill NaN values with 0
host_year_medal_countrys_df = host_year_medal_countrys_df.fillna(0)

# format values in all columns except Games to integers
host_year_medal_countrys_df = host_year_medal_countrys_df.astype(int)

host_year_medal_countrys_df

In [ ]:
#Sort the data by the total number of medals
host_year_sorted = host_year_medal_countrys_df.sort_values(by=('Medal', 'count'), ascending=False)

#Reset the index
host_year_sorted = host_year_sorted.reset_index()

#Fix the multi-index by removing the second level
host_year_sorted.columns = host_year_sorted.columns.droplevel(1)

#Rename the index to Rank
host_year_sorted.index.name = 'Rank'

#Start the index at 1
host_year_sorted.index = host_year_sorted.index + 1

#Change the column name from Medal to Total Medals
host_year_sorted = host_year_sorted.rename(columns = {"Medal": "Total Medals"})

# Move the Total Medals column to the end
host_year_sorted = host_year_sorted[['Country', 'Gold', 'Silver', 'Bronze', 'Total Medals']]

#Make Rank the first column
host_year_sorted = host_year_sorted.reset_index()

#View the dataframe
host_year_sorted

In [ ]:
#Create a table of the top 15 countries by total medals for a specific year

# Define a function to highlight the host country
def highlight(nation):
    if nation['Country'] == host_country:
        return ['background-color: #add8e6; color: black']*6
    else:
        return ['background-color: white; color: black; border-top: 1px solid black']*6
    
# Define styles to hide the index and apply border
styles = [
    #dict(selector="th", props=[("display", "none")]),  # Hide the header row
    dict(selector=".row_heading", props=[("display", "none")]),  # Hide the index column
    dict(selector=".blank", props=[("display", "none")]),  # Hide any blank cells
    dict(selector="tr", props=[("border", "1px solid black"), ("padding", "10px"), ("font-size", "10pt")]),  # Apply border, padding, and larger font
    #dict(selector="th", props=[("border", "1px solid black"), ("padding", "10px"), ("font-size", "10pt")]),  # Apply border, padding, and larger font
]    

# Create a styled dataframe
styled_df = host_year_sorted[['Rank', 'Country', 'Bronze', 'Silver', 'Gold', 'Total Medals']].iloc[:35] \
    .style \
    .set_caption('Medals by Country: Summer Olympic Games sorted by Gold Medals [Top 15]') \
    .set_table_styles(styles) \
    .bar(subset=['Gold'], color='#f0c05a') \
    .bar(subset=['Silver'], color='Lightgray') \
    .bar(subset=['Bronze'], color='#a97142') \
    .hide(axis="index").apply(highlight, axis=1)

# Save the styled dataframe to a html file
styled_df.to_html('resources/medals_by_country.html')

# Display the styled dataframe
styled_df

In [ ]:
#Create a new dataframe that only has your country as the host city
hosting_summer_df = summer_df[summer_df["City"] == host_city]
hosting_summer_df = hosting_summer_df[hosting_summer_df["NOC"] == host_NOC]

#Drop Unnecessary Columns
hosting_summer_medals_df = hosting_summer_df.drop(columns = ["Name", "Sex", "Age", "ID", "Height", "Weight","Games", "Sport", "Event"])

hosting_summer_medals_df = hosting_summer_medals_df.dropna(subset = ["Medal"]) 

#Reset the index
hosting_summer_medals_df = hosting_summer_medals_df.reset_index(drop=True)

#Group the date by year and medal
hosting_summer_medals_df = hosting_summer_medals_df.groupby(['Year', 'Medal']).size().unstack(fill_value=0)

#Sum the values in Bronze, Gold, and Silver into a Total Medals column
hosting_summer_medals_df['Total Medals'] = hosting_summer_medals_df.sum(axis=1)

#Give the average number of medals won by year
avg_hosting_summer_medals_df = hosting_summer_medals_df['Total Medals'].mean()

avg_hosting_summer_medals_df

In [ ]:
#Create a new dataframe that only has London as the host city
nonhosting_summer_df = summer_df[summer_df["City"] != host_city]
nonhosting_summer_df = nonhosting_summer_df[nonhosting_summer_df["NOC"] == host_NOC]

#Drop Unnecessary Columns
nonhosting_summer_medals_df = nonhosting_summer_df.drop(columns = ["Name", "Sex", "Age", "ID", "Height", "Weight","Games", "Sport", "Event"])

nonhosting_summer_medals_df = nonhosting_summer_medals_df.dropna(subset = ["Medal"]) 

#Reset the index
nonhosting_summer_medals_df = nonhosting_summer_medals_df.reset_index(drop=True)

#Group the date by year and medal
nonhosting_summer_medals_df = nonhosting_summer_medals_df.groupby(['Year', 'Medal']).size().unstack(fill_value=0)

#Sum the values in Bronze, Gold, and Silver into a Total Medals column
nonhosting_summer_medals_df['Total Medals'] = nonhosting_summer_medals_df.sum(axis=1)

#Give the average number of medals won by year
avg_nonhostingsummer_medals_df = nonhosting_summer_medals_df['Total Medals'].mean()

#View the dataframe
avg_nonhostingsummer_medals_df

In [ ]:
#Create an mlines chart to show the avg_hosting_summer_medals_df and avg_nonhosting_summer_medals_df
fig, ax = plt.subplots()
fig.set_size_inches(10, 6)

# Create a line for the average number of medals won in London
ax.axhline(y=avg_hosting_summer_medals_df, color='b', linestyle='-', label='Hosting Average Medals')

# Create a line for the average number of medals won in non-London
ax.axhline(y=avg_nonhostingsummer_medals_df, color='r', linestyle='-', label='Non-Hosting Average Medals')


# Set the title and labels
ax.set_title('Average Medals Won by [INSERT COUNTRY] in Summer Olympics')
ax.set_xlabel('Year')
ax.set_ylabel('Average Medals Won')

# Merge the two DataFrames into a single DataFrame that includes all years
all_years_df = pd.concat([hosting_summer_medals_df, nonhosting_summer_medals_df])

# Set the x-axis to be the years
x = all_years_df.index.unique()
ax.set_xticks(x)
ax.set_xticklabels(x, rotation=45)

# Plot the data
ax.plot(hosting_summer_medals_df.index, hosting_summer_medals_df['Total Medals'], label='Hosting Medals', marker='o')
ax.plot(nonhosting_summer_medals_df.index, nonhosting_summer_medals_df['Total Medals'], label='Non-Hosting Medals', marker='o')

# Add a legend
ax.legend()

# Save the figure
plt.savefig('resources/avg_medals.png')

# Show the plot
plt.show()